In [1]:
#pip install folium

In [ ]:
import folium
from folium.plugins import HeatMap
import pandas as pd
from datetime import datetime

# Constants for the map boundaries and grid dimensions
min_lat, max_lat = 35.225, 35.311
min_long, max_long = -120.71, -120.625
num_columns, num_rows = 12, 19  # From 'E' to 'P', inclusive; and '3' to '20', inclusive

# Function to generate grid dictionary with corrected latitude calculation
def generate_grid_dict():
    grid_dict = {}
    lat_step = (max_lat - min_lat) / (num_rows - 1)
    long_step = (max_long - min_long) / (num_columns - 1)
    for col in range(num_columns):
        for row in range(num_rows):
            inverted_row_index = (num_rows - 1) - row
            middle_lat = min_lat + lat_step * inverted_row_index + lat_step / 2
            middle_long = min_long + long_step * col + long_step / 2
            grid_id = f"{chr(ord('E') + col)}-{3 + row:02}"
            grid_dict[grid_id] = {'lat': middle_lat, 'long': middle_long}
    return grid_dict

grid_dict = generate_grid_dict()

# Initialize Map
initial_location = [35.2828, -120.6596]  # San Luis Obispo, California
map_zoom_start = 13
m = folium.Map(location=initial_location, zoom_start=map_zoom_start)

# Load your data
slopd_data = pd.read_csv('Full_Data_Final.csv')
slopd_data['Date'] = pd.to_datetime(slopd_data['Date'])

# Filter out specific rows if necessary (e.g., '911 Abandon' types)
slopd_data = slopd_data[slopd_data['Crime_Type'] != '911 Abandon']

# Group data by 'Type'
for type_, group in slopd_data.groupby('Crime_Type'):
    heat_data = []
    for _, row in group.iterrows():
        grid_id = row['Grid']
        if grid_id in grid_dict:
            location = grid_dict[grid_id]
            heat_data.append([location['lat'], location['long']])

    # Create a heatmap layer for each type and add it to the map
    if heat_data:  # Only add the layer if there is data
        HeatMap(heat_data, name=type_).add_to(m)

# Add LayerControl to toggle between types

combined_heat_data = []
for _, row in slopd_data.iterrows():
    grid_id = row['Grid']
    if grid_id in grid_dict:
        location = grid_dict[grid_id]
        combined_heat_data.append([location['lat'], location['long']])

# Create a combined heatmap layer and add it to the map
if combined_heat_data:  # Only add the layer if there is data
    HeatMap(combined_heat_data, name="All Crime Types").add_to(m)

# Add LayerControl to toggle the combined heatmap
folium.LayerControl().add_to(m)

# Display the map in a Jupyter notebook directly
m

# Or save the map to an HTML file and open it in a web browser
# m.save('heatmap_per_type.html')
